In [45]:
import sys
import pandas as pd
import numpy as np
import random
import requests
import json
import matplotlib.pyplot as plt
import time

In [46]:
#### get all csv dataframes for trending
US_trending_df = pd.read_csv('youtube-new/USvideos.csv') #USA
CA_trending_df = pd.read_csv('youtube-new/CAvideos.csv') #CANADA
DE_trending_df = pd.read_csv('youtube-new/DEvideos.csv') #GERMANY
FR_trending_df = pd.read_csv('youtube-new/FRvideos.csv') #FRANCE
GB_trending_df = pd.read_csv('youtube-new/GBvideos.csv') #GREAT BRITAIN
IN_trending_df = pd.read_csv('youtube-new/INvideos.csv') #INDIA

# JP_trending_df = pd.read_csv('youtube-new/JPvideos.csv', encoding="UTF-8") #JAPAN

# KR_trending_df = pd.read_csv('youtube-new/KRvideos.csv') #SOUTH KOREA

# MX_trending_df = pd.read_csv('youtube-new/MXvideos.csv') #MEXICO

# RU_trending_df = pd.read_csv('youtube-new/RUvideos.csv') #RUSSIA

list_of_all_trending_dfs = [US_trending_df, CA_trending_df, DE_trending_df, FR_trending_df, GB_trending_df, IN_trending_df]
full_trending_df = pd.concat(list_of_all_trending_dfs)

In [53]:
f = open("apiKey", "r")
key = f.read()
## for each videoId, find a related video
def do_search_youtube_request(videoId):
    url = "https://www.googleapis.com/youtube/v3/search?part=snippet&maxResults=50&order=relevance&relatedToVideoId={}&type=video&videoDefinition=any&key={}".format(videoId, key)
    r = requests.get(url)
    return r

## given a set of videoIds, find insights (statistics, tags, etc)
def find_video_insights(videoIds):
    print(videoIds)
    url = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id={}&key={}'.format(videoIds, key)
    r = requests.get(url)
    return r

## call this with 1 country at a time 
def process_youtube_requests(videoIds):
    df = pd.DataFrame(columns=['video_id', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 
                               'views', 'likes', 'dislikes', 'comment_count', 'description'])
    relatedVideoIds = []
    for videoId in videoIds:
        try: 
            response = do_search_youtube_request(videoId)
            time.sleep(2)
            if (response.status_code == 200):
                r1 = response.json()
                relatedVideoIdItems = r1['items']
                for id in relatedVideoIdItems:
                    relatedVideoId = id['id']['videoId']
                    relatedVideoIds.append(str(relatedVideoId))
            else:
                print(response.status_code)
        except:
            print ("Something went wrong here! 2")
    random.shuffle(relatedVideoIds)
    videoIdsForInsights = []
    for i in range(0, len(relatedVideoIds), 50):
        videoIdsForInsights.append(relatedVideoIds[i:i + n])
    for videoIdList in videoIdsForInsights:
        videoIdsStr = '%2C'.join([str(elem) for elem in videoIdList])
        r2 = find_video_insights(videoIdsStr)
        time.sleep(2)
        if (r2.status_code == 200):
            r = r2.json()
            i = 0
            while (i < len(videoIdList)):
                try:
                    id = videoIdList[i]
                    title = (r['items'][i]['snippet']['title'])
                    channel_title = (r['items'][i]['snippet']['channelTitle'])
                    category_id = (r['items'][i]['snippet']['categoryId'])
                    publish_time = (r['items'][i]['snippet']['publishedAt'])
                    tags = '|'.join((r['items'][i]['snippet']['tags']))
                    views = (r['items'][i]['statistics']['viewCount'])
                    likes = (r['items'][i]['statistics']['likeCount'])
                    dislikes = (r['items'][i]['statistics']['dislikeCount'])
                    comment_count = (r['items'][i]['statistics']['commentCount'])
                    description = (r['items'][i]['snippet']['description'])
                    data = {'video_id': id, 'title': title, 'channel_title': channel_title, 'category_id' : category_id,
                           'publish_time' : publish_time, 'tags' : tags, 'views' : views, 'likes' : likes, 'dislikes' : dislikes,
                           'comment_count' : comment_count, 'description' : description}
                    df = df.append(data, ignore_index = True)
                except:
                    print("Something went wrong! 3")
                i = i + 1
        else:
            print("Something went wrong! 4")
            print (r2.status_code)
            print(r2.text)
    return df
        

In [54]:
n = 50
US_trending_videoIds = US_trending_df.sample(n)['video_id'].tolist()
CA_trending_videoIds = CA_trending_df.sample(n)['video_id'].tolist()
DE_trending_videoIds = DE_trending_df.sample(n)['video_id'].tolist()
FR_trending_videoIds = FR_trending_df.sample(n)['video_id'].tolist()
GB_trending_videoIds = GB_trending_df.sample(n)['video_id'].tolist()
IN_trending_videoIds = IN_trending_df.sample(n)['video_id'].tolist()

In [ ]:
## do following requests separately with a new API Key

In [55]:
not_trending_us_df = process_youtube_requests(US_trending_videoIds)

oVEBZLrjpw4%2COlY_8ulFEuY%2C2sAoKmg7qPI%2C0SrHCUkKibQ%2CFLSn34AKRu8%2CIfZqHBtGzDs%2CAIQt9Jdp94Q%2Cf63BmPpHw8k%2C6mCfVaEljqU%2CuFOZb9nl5Yc%2CPZWlGd-9aqk%2C7nInqQ3QNo4%2CtFoFtIqvkGE%2CVIVF65-HTlc%2Ceq66zJi67Ew%2C2bNUyRCx2Yw%2C6mahB8pi_sc%2CRzzEFTHZ0dY%2CpaNVYGfGrqI%2CR4U-JgRx3iQ%2CNhTulKznx_Y%2CgAcvbGuqG0Q%2CDvwnuFhvlYA%2Cd2EmDuUISHk%2CkQ1xE4sK9Qk%2CZsE_myELN8I%2CEvHSWv424JI%2C1hi6m09ZWPo%2CFm_lRiHhgvo%2CqXGkWDrNNls%2CGM0IB-1xJ8s%2CoGBfkAQgIdo%2CkuOEpC4u6Tg%2C1hLWOcjXOM0%2CQtx9QEMy8sI%2CGdtUHwEe14Y%2CBtBrSH_UL7w%2CSqV5WEGUocU%2CnJ1JA6fAh5A%2CldhxemRcQ-c%2CZ7PW7C7raA8%2CVzt_necGWzw%2CKtA3Wp8eNgA%2C9m0B9qkeq4g%2C8_KBQEGWjow%2CDWnxgtEGTqw%2Czo5FRoSmjHc%2CyXQViqx6GMY%2CBkd0LxBd2L8%2CYmjuzSNAibw
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
BWY7EH9AFRM%2CeLFvA_ozB54%2C1Oqm6eO6deU%2CCnY2szI1Sv0%2CDzRuMFP-F18%2CPpgPcKYKl3A%2CPTK6Yr-St8s%2Cr7qovpFAGrQ%2CaBt8fN7mJNg%2CswSytFVMHuU%2CzrOd0SG0wAQ%2C4dD3U-xjlNU%2CP4hk1jPRSrM%2C

Something went wrong! 3
Something went wrong! 3
_lhibYD39Gs%2C-a4r9dsD2ig%2CNHozn0YXAeE%2CrnlzX2A4PGg%2CeDD9QiL9bFU%2CFeQUavm6XcY%2C9NkHIv25IH4%2CLUjn3RpkcKY%2C5EaF7m_KIzk%2CE7n7bMY4FD0%2CIl-an3K9pjg%2C-p6WWRarjNs%2CbKMrw4rqzdY%2CkxOYvI0pfLo%2CvoG07pt-KYI%2C5oX_gwwACls%2CYjUbHESi9Ag%2CMWCBwcVFCxE%2CTtR2ywaMgug%2C7Ve82ibSelM%2Cph8rSKRQDLI%2C-Yv3GUP73cU%2CPTKF1sm6ov4%2CVZPkIPfnzqs%2CQ2-Di8SIBm8%2C6KdviedF9iM%2C5SPyH9hl97o%2CmsrYgp0-4fY%2C8SdGgAAzZTg%2CjLntFKtR66g%2C-WIqbivXbho%2CdsRmALOKpbo%2CrdaxCBwiYoc%2CQm5X9nXyl8I%2CRQjqSQo5FY0%2CRps8nZFiYoA%2CvTnWFT3DvVA%2CylO0UB2Fmio%2C7C2z4GqqS5E%2ConFPBPzPd-c%2CMmSe-kIabNg%2CWMs3NOW1TDs%2C5clXuoK5Lm4%2Cwk0bgpN3GCo%2Ct9GNMVfEiG4%2COsKcXqjFxAk%2CAkq6YjbcL3s%2CgkAfjRolNCI%2CThvm6dADOms%2Crg1VufQmdak
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
zTmhDyxBybU%2C7IPGFzJrgZs%2Ct37cTzR7szw%2CJsB57FtaxXQ%2ChHl94LGqB6A%2CWTjddKXBNwQ%2CVIOToSgMQN0%2CqanOdP6oaMw%2CT5W3SHUuy7k%2Cos_K-e_qwu8%2C-Meu02KLMYE%2C1rS9

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
PH9vPCrDRU4%2CSuYmGnHn5ko%2Cl2StCpw8z-s%2C07vL6sW2XPs%2CUNAa5-uCowY%2CCYoPOJCeJvU%2CMjusk3sDBw8%2Ckufxf2JiYyQ%2CiQp1_GfDhwQ%2Cl9X4ip8M6zc%2CUSXoINPEhoA%2CzM9WVD0OfuA%2CdfnCAmr569k%2Cnw0KwRLHuaI%2CxRLQtkhq3So%2CNshiJSNBWNs%2CgzftYBJ4xSQ%2CN6D7BZnNso8%2CNWf8P-u_kjc%2CTW4unLbSiPQ%2Cw8okHnF7yF8%2CyHzPS7t8jeE%2CaYCFrVp1LIY%2CW6QoMsXLBOM%2CqlVnSFp_big%2CKET6qfLVcCQ%2ClydBPm2KRaU%2Cq7NrRw9SP3w%2CRsXuIT7x4Dk%2CNIc_vvMJ2xY%2Cqt6heoqqp3s%2CkWvs5H32Oyc%2ClOEqzt36JEM%2CHGqlvqN6X5Q%2CMc3B57g59Gw%2CJD9j-Xka6sA%2CJ-dv_DcDD_A%2CTYSB3IMTIKY%2CHDZXA6IWkIo%2CJ4WoY6uLy4E%2Cj0wNmlnHk4M%2CA70B_LKyOrU%2C1qpVkh61meM%2CQKyul7puruQ%2C2k2QSTIYK8E%2C7g51_tO6-eA%2CsYX0h7q_lO0%2COc265QiC4eM%2CsZvF9y9D8tc%2CSt5PLcxZy44
Something went wrong! 3
Something went wrong! 3
tpotfGS1KBY%2CyNXhYf-maeA%2CXp7t9fWz62Q%2CeamHg95uSXw%2COaaJqJ7EvhY%2CaEGO5sU3YqA%2CC95o0MRzBVs%2C-j99kU2xALk%2Cp0nMpHixbeM%2CQhSS1tsM

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
BWxzrUFX8y0%2CSmB6shj8qmE%2Ck1vE_LVBx4s%2CSvDsyyA3o2E%2CUlIe93oHUjo%2CeT9QeDqZbWc%2CZF27oDmQ7xo%2CFlbi7hiIFlI%2CN0TcE4fSOJI%2C9j9RrWgo5rE%2Cibeg1VLc0mA%2CuJeqMlKyFik%2C7W194GQ6fHI%2CzHg6RINhYJk%2C66zmsgOru-0%2C9F20yBjYNfY%2CoLOz7i04n0I%2CCWqbZ8K2Xhk%2CipDrhRWrfQw%2Cyt9qpIRIwE0%2ChVrwrI6yyjc%2CMswrWWgG-uc%2CZA5OkUqgrcI%2CEhSXsTSNDyo%2CwmVJyICw1C8%2CkVJvxD7Ldu8%2CcpxDZPl-FD4%2Cic6a8etIiFk%2CXGEbzK4jxyw%2C8_lFUGkur_8%2Ce0-ih37lcqI%2CIr0w28eR6bw%2CBsPoSkYhNC0%2Cy6ux0LCkDbQ%2CY9ALkBZ9N9M%2CuBzA9f7QnVE%2CngTLeDztGNc%2ClfEKnvdxvZs%2CSd2d0pbeMGg%2COINZX0SFVn8%2C1g4CiINEhRc%2CWNbRq-qco58%2CuCcNcHx2DpY%2CDVR-ZUVnseA%2CQGmIkreNmQ4%2CRspfMI-rZpw%2CzLUGcJQX9dc%2CTg1ZzRlEG6Y%2CJbMCGKsn1yI%2Cq8OVgc-xXlI
Something went wrong! 3
NmMybTkb9KA%2CMuIgkgl2w8A%2CABZ71Sp_qLM%2Cj4dbKvTUS20%2C1W_zM7koJy8%2CyH1oVW2lX8w%2CH4tg9enFEig%2C7bJUgXDpU7M%2Cx2u-KEex34A%2CDV7hop4m0YQ%2CLY13M5qQFT8%2Cwr8S-2jrmZs%2CBeHJHpHY4O8%2Cv7x_Yoq0Npw%2CTUmyygCMMG

OWBSb8BPqkQ%2CBs-Hr-gEJAU%2C6xUPaVfmcUs%2CgzWCaYxIo6U%2CTJnu8KEtCqY%2CoyDf_RKmDnE%2CjBkT5oLtFy4%2CLTR_YT3IFIE%2CmbEfzuCLoAQ%2CZh1iNlI_OQE%2CAmfNAaK23UU%2C2drssuwuM7c%2CFITSPSA8gQs%2CpXmQHiQmKs4%2CNDWqXHZARfA%2CEUoe7cf0HYw%2C0Gl2QnHNpkA%2C9Jj0p6RG0fo%2CyUM7YXnWLrg%2C8eobsHXCfys%2CCa3XvWlICus%2CHfSf_WDFgbk%2C07T8OXtGCbI%2CnHO50-7l-Vg%2CXXlXjAtm8H0%2CeGPvtsUsYdU%2C1WkjA4oCKn8%2CqtYBzY7nvzE%2C7AXo-bYGXI4%2CqeuaKPiSdwo%2C8DDU5X8KNyU%2CKr-iXEy8u1k%2CxYkfmqX0vqk%2CoiSgAiM0d8A%2CROYA4u7cuR4%2CcPYsxM3_3t0%2Cp5xIiIelscc%2CLR708uA4zQ8%2CM8ZAfykOLxU%2CYU8CvjIwAlY%2CTNu1201wy_U%2C1pmUSYfBmF0%2Cms6teOnlPzw%2CL8HQu_PqV7k%2C2kFSJE-nUP8%2C0SCsbGEkzXU%2CDUg3bgmSNEY%2CGIMVge5TYz4%2Cq9O9OjLlNik%2CQCdAX2_uHn0
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
PCkINFoQF2s%2CPrqcwH8LhDU%2CAhFVt5g6ugw%2Cbyr00d5QrLI%2CLZqgE_mE-fU%2CAydU3Gnj_VY%2C1iq_AP6WjUA%2CFg1zD2l8N6g%2CrllK0NBtk-w%2CnejGufWS5O0%2CYQQRPo3aXB8%2CMSP_nLMkCeA%2CJtyGQwohA80%2Cq9lbAH9ttS4%2CKAJqL-60a08%2CntCMoh-0ogo%2C8IvGpS

In [56]:
not_trending_us_df

,video_id,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,description
0,oVEBZLrjpw4,Carrie Underwood - Two Black Cadillacs (Offici...,carrieunderwoodVEVO,10,2013-01-23T23:55:09.000Z,carrie underwood Two Black Cadillacs live|acou...,38301587,223702,10633,13313,Carrie Underwood's official music video for 'T...
1,OlY_8ulFEuY,Tony Hawk Ramping Things Up,The Slow Mo Guys,24,2018-02-07T17:00:03.000Z,Super Slow Show|Slow Motion|4K|Explosion|Slo M...,1828410,62632,552,2032,Tony Hawk joins Dan and Gav to film a rare ska...
2,2sAoKmg7qPI,"GOT7 ""Girls Girls Girls"" M/V",jypentertainment,10,2014-01-14T23:00:08.000Z,GOT7_Girls Girls Girls_M/V|GOT7 Music Video|GO...,82730693,968532,16798,51986,"GOT7(갓세븐) ""Girls Girls Girls"" M/V\n\nDownload ..."
3,0SrHCUkKibQ,This is THE END of Call of Duty as we know it...,TmarTn,20,2018-04-19T17:19:53.000Z,call of duty|cod|black ops 4|bo4|overwatch|bla...,462344,10444,2402,3209,This is THE END. Perks replace killstreaks? Ov...
4,FLSn34AKRu8,RANKING EVERY GUN IN FORTNITE FROM WORST TO BEST!,TheSmithPlays,20,2018-01-06T19:06:30.000Z,TheSmithPlays|The|Smith|Plays|SmithPlays|ZHous...,5491881,71461,16685,37470,➤ Help Me Reach 1.8 Million Subscribers! https...
...,...,...,...,...,...,...,...,...,...,...,...
2170,1kunCCZfckM,JuJu Smith-Schuster Pranks Lil Yachty!,JuJu Smith-Schuster,24,2018-03-17T18:41:14.000Z,juju lil yachty|lil yachty|prank|juju smith-sc...,1113406,25721,428,1388,Lil Yachty came to the FaZe house and I pranke...
2171,GTW8IplsKmM,Kevin Hart Catches a High Eating Spicy Wings |...,First We Feast,26,2016-10-11T15:00:01.000Z,First we feast|fwf|firstwefeast|food|food porn...,23366704,288802,8193,15202,As his history-making comedy special What Now?...
2172,XljNJ3irnzk,Canadian and French People Swap Snacks,BuzzFeedVideo,22,2019-11-19T19:00:02.000Z,BuzzFeed|BuzzFeed Video|Canadian and French Pe...,115294,2491,85,468,"""I'm sorry Canada that I didn't pick any of yo..."
2173,Jx4YjzRh46w,How Extreme Is The Weather In Our Universe?,Life Noggin,1,2016-05-16T13:00:02.000Z,weather|space|space weather|jupiter|jupiter re...,931013,15007,191,1573,We love talking about the weather here on Eart...


In [57]:
not_trending_ca_df = process_youtube_requests(CA_trending_videoIds)

403
403
403
403
403
403
403
yi9HcBgDok0%2CT01rP733vxY%2C44GzNXRDAh8%2CYrlVffqU7sY%2CJLjH0rrNFeU%2Cz-9stNuZ42U%2Ce25q_MtDEhU%2C8BQH4vkS_Tk%2CkRo_9g0g9-k%2Cr_uDQwx-vKY%2CFC3y9llDXuM%2Czu_tFvfn2KU%2CagyTn10mW74%2CugAUMbceVNU%2C9GcOocjXB8k%2CnQl96NAbQlY%2ChK2IANTq6eo%2C-0bsHYdb1BI%2Ced7TTF_DkWQ%2CGL7Mo_B3O74%2CQOGJMF5ZPqA%2CI2O7blSSzpI%2COCJwHrvutGk%2CG01nkoEZ8UM%2COIklhU34oHI%2C3QiK2jOUuk0%2CC3a6DbxkqvQ%2CyEWC36R2iao%2CRolwE1lXOq8%2COxBJ2xjSFOQ%2CPceZkJyQqbo%2Cbx1Bh8ZvH84%2CU0GVhRpl6DQ%2CfuHcdFPBKy0%2CPIh2xe4jnpk%2CbqO7bZmBpUw%2C7Jvsj8cTS_g%2CrsFWbDl-ICw%2C5-uDhcnL07A%2C1G0sHY7gO54%2CQ0wbyQRRQJA%2Ctnzz-eFmKaw%2Cp6YLLytSdpQ%2CUrFsOWfxAQ4%2CEFAqIdftrZk%2CwCJYbuOmnbU%2CEv6R_SU-a9c%2ChWFDHynfl1E%2CMgBPQbXqN6c%2CZJR6JWHuMHw
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
nORg_aXMsmA%2CQbPEPnKOBT8%2CeG-nYkKJTgo%2CN5KUHEpiXU0%2CakA_Z9Xmnvc%2CxGPeNN9S0Fg%2ClEI60T2QmeQ%2CXpFWbADgtKI%2CJNyZ49q4vrU%2C4GcYzuDdi9E%2COdKcfmA664A%2C

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
h-k16VAt_1w%2CZSKEihd7Vlg%2CkA61fzKEIZE%2CS4utkb6vSyQ%2CXRATDHXxuqo%2CJZNgW8_1TVQ%2C5csoc55v2Ls%2C9hcutXKJgJY%2CSYccDd2zpJo%2CF2MFQ0O3twI%2CnXTI5asbLyU%2C01SEpNDUlgE%2CbCE2R_48pyU%2CVWcb6uwUoh8%2CRIZdjT1472Y%2CITidxU_QrsI%2CCDl9ZMfj6aE%2CGelnLHtIwxc%2COH-e5i_Ouh8%2CDzuOaJ9f8qI%2Ct8yUKzMnlFU%2CDtIS_j4VsXo%2C1C0Im1QoT68%2CpYDK6aeBFks%2CmhDNQ2rOQKw%2CMc6VA7Q1vXQ%2C2rYw3j-QDQE%2CKgBjJWNG1-Y%2CA6Wg5Rea9TE%2CJNcoYLVFCKg%2Ca4vT_DZwOUo%2C4mdhT0hMc3U%2CrOoYji5CGsM%2CWREtr-ZW5CQ%2CvNj0psXnS6w%2Cc4BLVznuWnU%2CWDAd0S92Uko%2CFwZvuV79jeY%2CaIvG8aCuYFw%2CNCFO5eifDtg%2C54vSt1L8nvw%2C_OWD06buI44%2CDAkV6EwDXN4%2CzxwfDlhJIpw%2CTHxkj29w3rA%2CHHHcA4lnIW0%2CMYXAZdxm-3w%2Ci-53VX8Z0bU%2CMtzMUJIofNg%2CYpYhGdrknlA
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
4Dl0Tqir8KQ%2C64OiI8yb7KE%2C8XBPYKIBfeo%2CaJOTlE1K90k%2Czb8f-yGildU%2CaCAhbjLKkt4%2CZJ3QQ-SNCso%2Cl_c2hqmFBdM%2C6DVsxSdSEbQ%2CxYu8Tdgc

Something went wrong! 4
403
{
 "error": {
  "errors": [
   {
    "domain": "youtube.quota",
    "reason": "quotaExceeded",
    "message": "The request cannot be completed because you have exceeded your \u003ca href=\"/youtube/v3/getting-started#quota\"\u003equota\u003c/a\u003e."
   }
  ],
  "code": 403,
  "message": "The request cannot be completed because you have exceeded your \u003ca href=\"/youtube/v3/getting-started#quota\"\u003equota\u003c/a\u003e."
 }
}

XODsQx1ddMU%2C6ked4EC_fE4%2CyynOMa34bpE%2C5Gvh-ybnAQc%2CpT_rXxA7cQk%2CASqCCBrCzKM%2Ctn9D7O_-j7o%2CTLZgdmGN2JQ%2CVMcrd8QAym0%2CK-4tVcAho74%2C7EhcBu7kOMM%2Coc0mR9v1_yE%2C4Dh-fBeqit0%2C5SBFZhLA6NI%2CXeZPM-fZymU%2C_d3fmShgl4I%2C17ZCwLQREsY%2CpxEP-qWEH40%2C7_fqF09QQjc%2CW9RcZ659HlQ%2Cm_IFikdUkHI%2COfKlSye7qEg%2CiK7ROzfOW8k%2C65AgPeDTU4U%2CAu9UVhvKhzs%2CuuwbnyTPoD8%2CXFCdyi3gUFk%2CvgD2iRhC99o%2CohXoXtxpOqg%2CWWbbob7PnM4%2C1VQ_3sBZEm0%2C6TXlYoJnsMY%2CZI9Fjo8k618%2CkQuIxhd1P40%2CwpU8I_XzyFo%2CCXVvjoPTn1A%2Cu5iWyNjU3Fo%2CINFIFUSD0EQ%2Ccv

Something went wrong! 4
403
{
 "error": {
  "errors": [
   {
    "domain": "youtube.quota",
    "reason": "quotaExceeded",
    "message": "The request cannot be completed because you have exceeded your \u003ca href=\"/youtube/v3/getting-started#quota\"\u003equota\u003c/a\u003e."
   }
  ],
  "code": 403,
  "message": "The request cannot be completed because you have exceeded your \u003ca href=\"/youtube/v3/getting-started#quota\"\u003equota\u003c/a\u003e."
 }
}

H-q03nwoLik%2Cex_sBZXU6iU%2CdK9UrsvCMWM%2C-TEIPVDYq-E%2CQRG5MOuPgwM%2C4IDLGUO7zXk%2CtJjLTadKLcY%2CX7EjObSF2Dg%2CnqyshkLRJvw%2CnznRgMaGna4%2CuuWOJOyTiqw%2CoFL5NoM9GVE%2CL3czjYgQiHs%2Cd_2rXTEYePg%2CdT2owtxkU8k%2CbEYSzS7bCZE%2CeeBQzjdlcLs%2CdMf8UEI067g%2CPPqGzczwseI%2CgqELqRCnW6g%2CQJlyLiuxUSw%2Cw-HZc0SGE2o%2CPBTrYJ9UIGA%2CPAczA2saYWM%2Cp4I1fP4DHqo%2C1Kr13Je3UD8%2C7BrTkSHtDEU%2CTz5L-rNWZBs%2CJfy981vKZjI%2Cyn5NjiE98tQ%2CkFSl-LTBIys%2CZOWMilTj-MM%2CFcNTEyEqxH0%2CAu9UVhvKhzs%2CIjlfQBQk_kg%2CHROuv4Z7Ejk%2C8giHvmeQKpY%2CQCuKbCxNTOY%2C6y

Something went wrong! 4
403
{
 "error": {
  "errors": [
   {
    "domain": "youtube.quota",
    "reason": "quotaExceeded",
    "message": "The request cannot be completed because you have exceeded your \u003ca href=\"/youtube/v3/getting-started#quota\"\u003equota\u003c/a\u003e."
   }
  ],
  "code": 403,
  "message": "The request cannot be completed because you have exceeded your \u003ca href=\"/youtube/v3/getting-started#quota\"\u003equota\u003c/a\u003e."
 }
}

4GcYzuDdi9E%2CNGtt7GJ1uiM%2CO1rnPPKSvq8%2CCxxS_vCSOfk%2CVqFeLOIue6o%2C56YfVbRigMQ%2CFCL9n8mMqJw%2CgsH2sMjXIiU%2CVgsmsZXMflk%2ChIAzDGwuzQo%2CQ2hqcPnIWiQ%2COzKZODBoDvg%2CGvFl6nl0rQs%2CAI6V2RhKBwg%2CWMxSdxsUpUY%2CE9s2l5fR_Mc%2COtL1fEEtLaA%2C1AyfBJMnv58%2C1cvgoY0cmzk%2C1vrEljMfXYo%2CrgCIs8KOL7c%2CHz2QkVCVj2E%2CJfQ3W0wjGh4%2C9y6Z5BLom80%2Czc8hbSM1zVo%2CZW4GHf-dfQU%2CcL8sG3SQmiw%2CedBM-ifCD8A%2CslZKqXY-k-U%2CuVTfszppJl8%2CerG5rgNYSdk%2C-GX5LrmT_ec%2CQsur4kxI9wQ%2CwWD3RCKe0k8%2CwfzpKFtWh2E%2CzrSsClUeT9w%2Ca0xSuBbHX5k%2Ch7M--4zIug4%2Cnz

In [ ]:
not_trending_ca_df

In [ ]:
not_trending_de_df = process_youtube_requests(DE_trending_videoIds)

In [ ]:
not_trending_fr_df = process_youtube_requests(FR_trending_videoIds)

In [ ]:
not_trending_gb_df = process_youtube_requests(GB_trending_videoIds)

In [ ]:
not_trending_in_df = process_youtube_requests(IN_trending_videoIds)

In [ ]:
list_of_all_nontrending_dfs = [non_trending_us_df, not_trending_ca_df, not_trending_de_df, 
                              not_trending_fr_df, not_trending_gn_df, not_trending_in_df]
full_nontrending_df = pd.concat(list_of_all_nontrending_dfs)

In [78]:
def insert_category_field(df, json_file):
    # add a category column using the category id from the json file
    id_to_category = {}
    with open(json_file, 'r') as f:
        data = json.load(f)
        for category in data['items']:
            id_to_category[category['id']] = category['snippet']['title']
        df.insert(4, 'category', us['category_id'].map(id_to_category))

In [ ]:
## insert new category field into dataframes
insert_category_field(US_trending_df, "US_category_id.json")
insert_category_field(CA_trending_df, "CA_category_id.json")
insert_category_field(DE_trending_df, "DE_category_id.json")
insert_category_field(FR_trending_df, "FR_category_id.json")
insert_category_field(GB_trending_df, "GB_category_id.json")
insert_category_field(IN_trending_df, "IN_category_id.json")

insert_category_field(not_trending_us_df, "US_category_id.json")
insert_category_field(not_trending_ca_df, "CA_category_id.json")
insert_category_field(not_trending_de_df, "DE_category_id.json")
insert_category_field(not_trending_fr_df, "FR_category_id.json")
insert_category_field(not_trending_gb_df, "GB_category_id.json")
insert_category_field(not_trending_in_df, "IN_category_id.json")

In [ ]:
## do following for both trending and nontrending datasets and/or per country
## do good visualizations for all!!!

## find likes to dislikes ratio ---> shows where people have the greatest divide and where people agree the most 
## which categories are the most popular? do highest average amongst likes and views ----> shows which type of videos people enjoy the most
## most trending videos in each country? do average amongst likes, views ------> shows most trending videos per country
## how long did a video stay trending? how long did it take to become trending?
## which country has the most active participation and online presence across trending and nontrending? ----> min, max, std, quartiles, counts, etc
## correlation between views, likes, dislikes, and comments
## get most frequent tags ----> run sentiment analysis on tags, run LDA model on tags
## what makes a video trending? find the minimum number/qualities required to make a video trending given our data
    ## -> for ex: need 2000 views, 1000 likes, 1500 comments, needs to be about sports, and the tags have to give off a "happy" feeling